# Fiverr Scraping Projet

## Explication


Nous avons décider de partir sur un projet d'analyze de données sur Fiverr, un site où des personnes proposent tout type de services à des utilisateurs. La première étape de ce projet fut de trouver le moyen de récupérer le plus d'informations possibles du site web grâce à la technique de scraping. Le temps de réalisation de cette étape dépend entièrement de l'éthique qu'ont eus les développeurs pour la réalisation dudit site. Et ce jeu de hasard, malheuresement, nous l'avons perdu.

![](./images/1.png)

on passe de \<b> à \<strong> pour aucune raisons pour 2 éléments différents d'une même liste

![](./images/2.png)

ou encore l'organisation des informations qui est très mal gérée

mais bref, juste quelques difficultés incontournables.

Nous avons eu 2 idées de script :  
    
- on choisi un sujet, photoshop, python ou piano, et pour chaque page, on récupères toutes les propositions de service que nous renvoie le site. Chaque page contient environ 40 propositions, et Fiverr nous donne des informations complémentaires comme la note moyenne de l'auteur, le nombre d'avis, la description et le lien du profil. Nous extrayons tout pour analyzer ces informations, pour les mettre en base de données. L'idée est de comparer toutes les propositions entre elles et de retourner la proposition la plus efficace suivant la note et le nombre d'avis.
- Depuis la base de données, on scrape tous les profils grâce au lien qu'on a scrapé dans le 1er script. Avec ces informations supplémentaires, nous avons eu l'idée de réaliser un profiler, grâce à un model de machine learning, on lui donne toutes ces informations et ils nous retourne le profil qui à le plus de chance de réussir

## Code

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
import pandas as pd
import re
import os
import jellyfish
from collections import Counter
import time

Choix du sujet et création du dossier qui contiendra les fichiers csv, notre base de données

In [ ]:
# ["python", "data science", "copyright", "comptability", "design", "excel, javascript", "adobe premiere", "photoshop", " c++", "guitare", "math", "piano", "violon", "java"]
subject = 'photoshop'

try: os.mkdir('./data/' + subject)
except FileExistsError: pass

Cette fonction retourne le driver obligatoire pour récuperer le code html de la page web, les options permettent d'éviter de se faire détecter par les différents bots (les sites web utilisent des techniques pour empêcher le scraping)

In [ ]:
def get_driver():
    options = Options()
    options.add_argument("--incognito")
    options.add_argument("--headless")
    options.add_argument('--disable-browser-side-navigation')
    return webdriver.Firefox(options=options)

Le script pour scraper les pages d'un sujet choisi et écrire dans un .csv les informations récupérées

In [ ]:
with open(f'./data/{subject}/cards.csv', 'a') as f:
    if f.tell() == 0:
        f.write("name;price;nb_comments;note;profil_link;description")
    for p in range(10):
        driver = get_driver()
        driver.get(f"https://fr.fiverr.com/search/gigs?query={subject}&source=pagination&search_in=everywhere&search-autocomplete-original-term={subject}&page={p}")

        if 'block.fiverr.com' in driver.current_url or '/404' in driver.current_url:
            print('blocked')
            raise Exception

        elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'gig-card-layout')]")
        for i, element in enumerate(elements):
            try: 
                name = element.find_element(By.XPATH, ".//div[contains(@class, 'seller-name')]/a").text
                profil_link = element.find_element(By.XPATH, ".//div[contains(@class, 'seller-name')]/a").get_attribute('href')
                description = element.find_element(By.XPATH, ".//h3/a").text.replace(';', ',')
                ratingText = element.find_element(By.XPATH, ".//span[contains(@class, 'gig-rating')]").text
                groups = re.match(r'(\d)(?:,)(\d)(?:\()(\d+)(?:\))', ratingText).groups()
                note = int(groups[0])+int(groups[1])/10
                price = int(element.find_element(By.XPATH, ".//a[contains(@class, 'price')]/span").text[:-2])/100
                
            except Exception as e:
                print(name)
                    
            f.write(f"{name};{price};{int(groups[2])};{note};{profil_link};{description}\n")
            
        driver.close()


On récupère ces informations

In [ ]:
cards = pd.read_csv(f'./data/{subject}/cards.csv', sep=';')

Le script pour scraper tous les profils et écrire dans un .csv toutes leurs informations

In [ ]:
with open(f'./data/{subject}/profils.csv', 'a') as f:
    if f.tell() == 0:
        f.write(f"name;location;created_at;response_time;last_order:languages;linked_acc;skills;education;description\n")

    for id in range(30):
        driver = get_driver()
        driver.get(cards.loc[id]['profil_link'])
        # The website can block from request or the page doesn't exist or the profil link doesn't exist so it returns to the main page
        if 'block.fiverr.com' in driver.current_url or '/404' in driver.current_url or len(driver.current_url) < 25:
            raise Exception

        name = cards.loc[id]['name']
        stats = [i.text.strip() for i in driver.find_elements(By.XPATH, "//ul[contains(@class, 'user-stats')]/li/*[self::b or self::strong]")]
        location = stats[0]
        created_at = stats[1]
        response_time = stats[2]
        last_order = stats[3]
        languages = [f"{i.text[i.text.find('(')+1:i.text.find(')')]}-{i.text.split('-')[-1].strip()}" for i in driver.find_elements(By.XPATH, "//div[contains(@class, 'languages')]/ul/li")]
        linked_acc = [i.text for i in driver.find_elements(By.XPATH, "//div[contains(@class, 'linked-accounts')]/ul/li/span[@class='text']")]
        skills = [i.text for i in driver.find_elements(By.XPATH, "//div[contains(@class, 'skills')]/ul/li/a")]
        try:
            education = driver.find_element(By.XPATH, "//div[contains(@class, 'education-list')]/ul/li/p").text
        except Exception:
            education = ''
        description = driver.find_element(By.XPATH, "//div[contains(@class, 'description')]/p").text.replace(';', ',')
        
        f.write(f"{name};{location};{created_at};{response_time};{last_order};{'|'.join(languages)};{'|'.join(linked_acc)};{'|'.join(skills)};{education};{description}\n")
        
        driver.close()

On récupère ces informations

In [ ]:
profils = pd.read_csv(f'./data/{subject}/profils.csv', sep=';')

Cette function permet de compter toutes les skills de tous les profils pour savoir quels compétences sont les plus demandées

In [ ]:
def get_common_skills(l: list, nb: int):
    skills = Counter({})
    for row in l:
        temp = row.split('|')
        for i in temp:
            skills[i] += 1
    return skills.most_common(nb)

In [ ]:
get_common_skills(list(profils['skills']), 5)